<h1>DATA CLEANING<h1>

Import libraries, read csv, drop useless columns and look at the dataframe

In [129]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd

In [2]:
presel=pd.read_csv('data/1976-2020-president.csv')
presel.drop(columns=['office','version','notes'],inplace=True)

In [3]:
presel.head()

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
0,1976,ALABAMA,AL,1,63,41,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,"FORD, GERALD",REPUBLICAN,False,504070,1182850,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,False,9198,1182850,OTHER
3,1976,ALABAMA,AL,1,63,41,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,False,6669,1182850,OTHER
4,1976,ALABAMA,AL,1,63,41,"HALL, GUS",COMMUNIST PARTY USE,False,1954,1182850,OTHER


In [12]:
presel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4287 entries, 0 to 4286
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              4287 non-null   int64 
 1   state             4287 non-null   object
 2   state_po          4287 non-null   object
 3   state_fips        4287 non-null   int64 
 4   state_cen         4287 non-null   int64 
 5   state_ic          4287 non-null   int64 
 6   candidate         4000 non-null   object
 7   party_detailed    3831 non-null   object
 8   writein           4284 non-null   object
 9   candidatevotes    4287 non-null   int64 
 10  totalvotes        4287 non-null   int64 
 11  party_simplified  4287 non-null   object
dtypes: int64(6), object(6)
memory usage: 402.0+ KB


Look at the party_simplified column

In [4]:
presel.party_simplified.value_counts()

party_simplified
OTHER          2524
DEMOCRAT        615
REPUBLICAN      613
LIBERTARIAN     535
Name: count, dtype: int64

Let's count Libertarian party candidates as 'OTHER'

In [5]:
presel.party_simplified = np.where(presel.party_simplified=="LIBERTARIAN", 'OTHER', presel.party_simplified)

In [6]:
presel.party_simplified.value_counts()

party_simplified
OTHER         3059
DEMOCRAT       615
REPUBLICAN     613
Name: count, dtype: int64

Get a list of all candidates that ran for the two major parties over the years

In [8]:
major_candidates=presel[presel.party_simplified!='OTHER'].candidate.unique()
major_candidates

array(['CARTER, JIMMY', 'FORD, GERALD', 'REAGAN, RONALD',
       'MONDALE, WALTER', 'BUSH, GEORGE H.W.', 'DUKAKIS, MICHAEL',
       'CLINTON, BILL', 'DOLE, ROBERT', 'BUSH, GEORGE W.', 'GORE, AL',
       'KERRY, JOHN', 'OTHER', 'MCCAIN, JOHN', 'OBAMA, BARACK H.',
       'ROMNEY, MITT', 'MITT, ROMNEY', 'TRUMP, DONALD J.',
       'CLINTON, HILLARY', nan, 'BIDEN, JOSEPH R. JR'], dtype=object)

Few issues here: 'OTHER' and null names, Mitt Romney's name written in two different ways.


Let's look at rows with null names for a major party

In [15]:
presel[(presel.candidate.isnull()) & (presel.party_simplified!='OTHER')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
3414,2016,ARIZONA,AZ,4,86,61,NaN,DEMOCRAT,True,42,2573165,DEMOCRAT


I will count these votes as protest votes, not for Hillary Clinton.

We can just get rid of all candidates with null names.

In [19]:
presel=presel[~(presel.candidate.isnull())]

In [24]:
presel.reset_index(drop=True,inplace=True)

Look at where Romney's name was written in a different way

In [25]:
presel[presel.candidate=='MITT, ROMNEY']

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
3148,2012,WASHINGTON,WA,53,91,73,"MITT, ROMNEY",REPUBLICAN,False,1290670,3125516,REPUBLICAN


In [26]:
presel[(presel.year==2012) & (presel.state_po=='WA')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
3147,2012,WASHINGTON,WA,53,91,73,"OBAMA, BARACK H.",DEMOCRAT,False,1755396,3125516,DEMOCRAT
3148,2012,WASHINGTON,WA,53,91,73,"MITT, ROMNEY",REPUBLICAN,False,1290670,3125516,REPUBLICAN
3149,2012,WASHINGTON,WA,53,91,73,"JOHNSON, GARY",LIBERTARIAN,False,42202,3125516,OTHER
3150,2012,WASHINGTON,WA,53,91,73,"STEIN, JILL",GREEN,False,20928,3125516,OTHER
3151,2012,WASHINGTON,WA,53,91,73,"GOODE, VIRGIL HAMLIN, JR.",CONSTITUTION PARTY,False,8851,3125516,OTHER
3152,2012,WASHINGTON,WA,53,91,73,"ANDERSON, ROSS CARL """"ROCKY""""",JUSTICE,False,4946,3125516,OTHER
3153,2012,WASHINGTON,WA,53,91,73,"LINDSAY, PETA",SOCIALISM AND LIBERATION PARTY,False,1318,3125516,OTHER
3154,2012,WASHINGTON,WA,53,91,73,"HARRIS, JAMES",SOCIALIST WORKERS,False,1205,3125516,OTHER


So they just made an error writing down Romney's name for the election in Washington in 2012.

Let's correct it.

In [31]:
presel.iloc[(3148,6)]='ROMNEY, MITT'

In [32]:
presel[presel.candidate=='MITT, ROMNEY']

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified


Look at the 'OTHER' candidates for a major party

In [34]:
presel[(presel.candidate=='OTHER') & (presel.party_simplified!='OTHER')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
2391,2004,MARYLAND,MD,24,52,52,OTHER,DEMOCRAT,True,7,2384238,DEMOCRAT


Again, I will count this as a protest vote.

Let's mark the party as 'OTHER'.

In [37]:
presel.iloc[(2391,-1)]='OTHER'

In [38]:
presel[(presel.candidate=='OTHER') & (presel.party_simplified!='OTHER')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified


In [39]:
major_candidates=presel[presel.party_simplified!='OTHER'].candidate.unique()
major_candidates

array(['CARTER, JIMMY', 'FORD, GERALD', 'REAGAN, RONALD',
       'MONDALE, WALTER', 'BUSH, GEORGE H.W.', 'DUKAKIS, MICHAEL',
       'CLINTON, BILL', 'DOLE, ROBERT', 'BUSH, GEORGE W.', 'GORE, AL',
       'KERRY, JOHN', 'MCCAIN, JOHN', 'OBAMA, BARACK H.', 'ROMNEY, MITT',
       'TRUMP, DONALD J.', 'CLINTON, HILLARY', 'BIDEN, JOSEPH R. JR'],
      dtype=object)

Now it's a proper list of all major party candidates from 1976 to 2020.

Let's get rid of all the other candidates.

In [41]:
presel=presel[presel.candidate.isin(major_candidates)]
presel.reset_index(drop=True,inplace=True)

In [42]:
presel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              1253 non-null   int64 
 1   state             1253 non-null   object
 2   state_po          1253 non-null   object
 3   state_fips        1253 non-null   int64 
 4   state_cen         1253 non-null   int64 
 5   state_ic          1253 non-null   int64 
 6   candidate         1253 non-null   object
 7   party_detailed    1252 non-null   object
 8   writein           1253 non-null   object
 9   candidatevotes    1253 non-null   int64 
 10  totalvotes        1253 non-null   int64 
 11  party_simplified  1253 non-null   object
dtypes: int64(6), object(6)
memory usage: 117.6+ KB


In [ ]:
presel.writein.value_counts()

Let's look at the write-in candidates.

In [44]:
presel[presel.writein==True]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
1085,2016,MARYLAND,MD,24,52,52,"TRUMP, DONALD J.",REPUBLICAN,True,259,2781446,REPUBLICAN
1086,2016,MARYLAND,MD,24,52,52,"CLINTON, HILLARY",DEMOCRAT,True,78,2781446,DEMOCRAT
1166,2020,DISTRICT OF COLUMBIA,DC,11,53,55,"BIDEN, JOSEPH R. JR",DEMOCRAT,True,317323,344356,DEMOCRAT
1167,2020,DISTRICT OF COLUMBIA,DC,11,53,55,"TRUMP, DONALD J.",REPUBLICAN,True,18586,344356,REPUBLICAN
1210,2020,NEW HAMPSHIRE,NH,33,12,4,"ROMNEY, MITT",NaN,True,170,806182,OTHER


Romney didn't run in 2020, so, again, these are protest votes.

Let's delete this data.

In [46]:
presel.drop(1210,inplace=True)

In [48]:
presel[(presel.year==2020) & (presel.state_po=='DC')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
1166,2020,DISTRICT OF COLUMBIA,DC,11,53,55,"BIDEN, JOSEPH R. JR",DEMOCRAT,True,317323,344356,DEMOCRAT
1167,2020,DISTRICT OF COLUMBIA,DC,11,53,55,"TRUMP, DONALD J.",REPUBLICAN,True,18586,344356,REPUBLICAN


For some reason, they flagged both major candidates as write-ins in DC in 2020. We can ignore that.

In [49]:
presel[(presel.year==2016) & (presel.state_po=='MD')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
1083,2016,MARYLAND,MD,24,52,52,"CLINTON, HILLARY",DEMOCRAT,False,1677928,2781446,DEMOCRAT
1084,2016,MARYLAND,MD,24,52,52,"TRUMP, DONALD J.",REPUBLICAN,False,943169,2781446,REPUBLICAN
1085,2016,MARYLAND,MD,24,52,52,"TRUMP, DONALD J.",REPUBLICAN,True,259,2781446,REPUBLICAN
1086,2016,MARYLAND,MD,24,52,52,"CLINTON, HILLARY",DEMOCRAT,True,78,2781446,DEMOCRAT


But in Maryland in 2016 only some votes for major candidates are write-in.

Let's add them to the non-write-in totals and delete the write-in rows.

In [55]:
presel.iloc[(1083,-3)]+=presel.iloc[(1086,-3)]
presel.iloc[(1084,-3)]+=presel.iloc[(1085,-3)]

In [57]:
presel.drop([1085, 1086],inplace=True)

In [58]:
presel[(presel.year==2016) & (presel.state_po=='MD')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
1083,2016,MARYLAND,MD,24,52,52,"CLINTON, HILLARY",DEMOCRAT,False,1678006,2781446,DEMOCRAT
1084,2016,MARYLAND,MD,24,52,52,"TRUMP, DONALD J.",REPUBLICAN,False,943428,2781446,REPUBLICAN


In [59]:
presel[presel.writein==True]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,writein,candidatevotes,totalvotes,party_simplified
1166,2020,DISTRICT OF COLUMBIA,DC,11,53,55,"BIDEN, JOSEPH R. JR",DEMOCRAT,True,317323,344356,DEMOCRAT
1167,2020,DISTRICT OF COLUMBIA,DC,11,53,55,"TRUMP, DONALD J.",REPUBLICAN,True,18586,344356,REPUBLICAN


Now we can just drop the writein column

In [61]:
presel.reset_index(drop=True,inplace=True)
presel.drop(columns='writein',inplace=True)

In [62]:
presel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              1250 non-null   int64 
 1   state             1250 non-null   object
 2   state_po          1250 non-null   object
 3   state_fips        1250 non-null   int64 
 4   state_cen         1250 non-null   int64 
 5   state_ic          1250 non-null   int64 
 6   candidate         1250 non-null   object
 7   party_detailed    1250 non-null   object
 8   candidatevotes    1250 non-null   int64 
 9   totalvotes        1250 non-null   int64 
 10  party_simplified  1250 non-null   object
dtypes: int64(6), object(5)
memory usage: 107.6+ KB


Let's look at the party_detailed column

In [63]:
presel.party_detailed.value_counts()

party_detailed
REPUBLICAN                 612
DEMOCRAT                   609
CONSERVATIVE                11
LIBERAL PARTY                6
WORKING FAMILIES             5
DEMOCRATIC-FARMER-LABOR      3
RIGHT-TO-LIFE                1
FREEDOM                      1
INDEPENDENCE                 1
WOMEN'S EQUALITY             1
Name: count, dtype: int64

In [64]:
presel[presel.party_detailed=='CONSERVATIVE']

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
66,1976,NEW YORK,NY,36,21,13,"FORD, GERALD",CONSERVATIVE,274878,6668262,OTHER
170,1980,NEW YORK,NY,36,21,13,"REAGAN, RONALD",CONSERVATIVE,256131,6201959,OTHER
273,1984,NEW YORK,NY,36,21,13,"REAGAN, RONALD",CONSERVATIVE,288244,6806810,OTHER
377,1988,NEW YORK,NY,36,21,13,"BUSH, GEORGE H.W.",CONSERVATIVE,243457,6485683,OTHER
481,1992,NEW YORK,NY,36,21,13,"BUSH, GEORGE H.W.",CONSERVATIVE,177000,7079432,OTHER
586,1996,NEW YORK,NY,36,21,13,"DOLE, ROBERT",CONSERVATIVE,183392,6439129,OTHER
691,2000,NEW YORK,NY,36,21,13,"BUSH, GEORGE W.",CONSERVATIVE,144797,6960215,OTHER
796,2004,NEW YORK,NY,36,21,13,"BUSH, GEORGE W.",CONSERVATIVE,155574,7448266,OTHER
900,2008,NEW YORK,NY,36,21,13,"MCCAIN, JOHN",CONSERVATIVE,170475,7722019,OTHER
1005,2012,NEW YORK,NY,36,21,13,"ROMNEY, MITT",CONSERVATIVE,262035,7116784,OTHER


In [67]:
presel[presel.party_detailed=='WORKING FAMILIES']

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
692,2000,NEW YORK,NY,36,21,13,"GORE, AL",WORKING FAMILIES,88395,6960215,OTHER
797,2004,NEW YORK,NY,36,21,13,"KERRY, JOHN",WORKING FAMILIES,133525,7448266,OTHER
902,2008,NEW YORK,NY,36,21,13,"OBAMA, BARACK H.",WORKING FAMILIES,159613,7722019,OTHER
1006,2012,NEW YORK,NY,36,21,13,"OBAMA, BARACK H.",WORKING FAMILIES,147643,7116784,OTHER
1110,2016,NEW YORK,NY,36,21,13,"CLINTON, HILLARY",WORKING FAMILIES,140041,7802084,OTHER


So there are still plenty of votes for the major party candidates, but not for the major parties themselves.

Let's look at what states are they from.

In [70]:
presel[~(presel.party_detailed.isin(['REPUBLICAN', 'DEMOCRAT']))].state.value_counts()

state
NEW YORK     26
MINNESOTA     3
Name: count, dtype: int64

Thanks, Minnesota and especially New York.

Let's look at the Minnesota rows.

In [71]:
presel[~(presel.party_detailed.isin(['REPUBLICAN', 'DEMOCRAT'])) & (presel.state=='MINNESOTA')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
671,2000,MINNESOTA,MN,27,41,33,"GORE, AL",DEMOCRATIC-FARMER-LABOR,1168266,2438685,DEMOCRAT
776,2004,MINNESOTA,MN,27,41,33,"KERRY, JOHN",DEMOCRATIC-FARMER-LABOR,1445014,2828387,DEMOCRAT
985,2012,MINNESOTA,MN,27,41,33,"OBAMA, BARACK H.",DEMOCRATIC-FARMER-LABOR,1546167,2936561,DEMOCRAT


In [73]:
presel[(presel.year.isin([2000, 2004, 2012])) & (presel.state_po=='MN')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
671,2000,MINNESOTA,MN,27,41,33,"GORE, AL",DEMOCRATIC-FARMER-LABOR,1168266,2438685,DEMOCRAT
672,2000,MINNESOTA,MN,27,41,33,"BUSH, GEORGE W.",REPUBLICAN,1109659,2438685,REPUBLICAN
776,2004,MINNESOTA,MN,27,41,33,"KERRY, JOHN",DEMOCRATIC-FARMER-LABOR,1445014,2828387,DEMOCRAT
777,2004,MINNESOTA,MN,27,41,33,"BUSH, GEORGE W.",REPUBLICAN,1346695,2828387,REPUBLICAN
985,2012,MINNESOTA,MN,27,41,33,"OBAMA, BARACK H.",DEMOCRATIC-FARMER-LABOR,1546167,2936561,DEMOCRAT
986,2012,MINNESOTA,MN,27,41,33,"ROMNEY, MITT",REPUBLICAN,1320225,2936561,REPUBLICAN


So, in 2000, 2004 and 2012 in Minnesota the Democratic party is just named differently. We can ignore that.

In [77]:
presel[(presel.year==2016) & (presel.state_po=='NY')]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
1107,2016,NEW YORK,NY,36,21,13,"CLINTON, HILLARY",DEMOCRAT,4379789,7802084,DEMOCRAT
1108,2016,NEW YORK,NY,36,21,13,"TRUMP, DONALD J.",REPUBLICAN,2527142,7802084,REPUBLICAN
1109,2016,NEW YORK,NY,36,21,13,"TRUMP, DONALD J.",CONSERVATIVE,292392,7802084,OTHER
1110,2016,NEW YORK,NY,36,21,13,"CLINTON, HILLARY",WORKING FAMILIES,140041,7802084,OTHER
1111,2016,NEW YORK,NY,36,21,13,"CLINTON, HILLARY",WOMEN'S EQUALITY,36294,7802084,OTHER


But we can't ignore these votes in New York.

Let's get a grouping of all candidate names per year per party.

In [92]:
candidates=presel[presel.party_simplified!='OTHER'].groupby(['year','party_simplified']).candidate.first()

candidates

year  party_simplified
1976  DEMOCRAT                  CARTER, JIMMY
      REPUBLICAN                 FORD, GERALD
1980  DEMOCRAT                  CARTER, JIMMY
      REPUBLICAN               REAGAN, RONALD
1984  DEMOCRAT                MONDALE, WALTER
      REPUBLICAN               REAGAN, RONALD
1988  DEMOCRAT               DUKAKIS, MICHAEL
      REPUBLICAN            BUSH, GEORGE H.W.
1992  DEMOCRAT                  CLINTON, BILL
      REPUBLICAN            BUSH, GEORGE H.W.
1996  DEMOCRAT                  CLINTON, BILL
      REPUBLICAN                 DOLE, ROBERT
2000  DEMOCRAT                       GORE, AL
      REPUBLICAN              BUSH, GEORGE W.
2004  DEMOCRAT                    KERRY, JOHN
      REPUBLICAN              BUSH, GEORGE W.
2008  DEMOCRAT               OBAMA, BARACK H.
      REPUBLICAN                 MCCAIN, JOHN
2012  DEMOCRAT               OBAMA, BARACK H.
      REPUBLICAN                 ROMNEY, MITT
2016  DEMOCRAT               CLINTON, HILLARY
      REPUB

In [99]:
candidates[2000,'DEMOCRAT']

'GORE, AL'

Now let's compute sums of votes in New York for all candidates and all years, 
then create a DataFrame with appropriate values.

In [105]:
# better to append to a list
ny_total_votes_list=[]
# look just at the New York data
ny_votes=presel[presel.state=='NEW YORK']
# list of all years
years=ny_votes.year.unique().tolist()

for year in years:
    # sum of votes for the Dem candidate this year
    dem_votes=ny_votes[(ny_votes.year==year) & 
                       (ny_votes.candidate==candidates[year,'DEMOCRAT'])].candidatevotes.sum()

    # sum of votes for the Rep candidate this year
    rep_votes=ny_votes[(ny_votes.year==year) & 
                       (ny_votes.candidate==candidates[year,'REPUBLICAN'])].candidatevotes.sum()

    # append appropriate rows as dictionaries to the list
    ny_total_votes_list.append( {
        'year':year,
        'state':'NEW YORK',
        'state_po':'NY',
        'state_fips':36,
        'state_cen':21,
        'state_ic':13,
        'candidate':candidates[year,'DEMOCRAT'],
        'party_detailed':'DEMOCRAT',
        'candidatevotes':dem_votes,
        'totalvotes':ny_votes[ny_votes.year==year].totalvotes.iloc[0],
        'party_simplified':'DEMOCRAT'
    })

    ny_total_votes_list.append( {
        'year':year,
        'state':'NEW YORK',
        'state_po':'NY',
        'state_fips':36,
        'state_cen':21,
        'state_ic':13,
        'candidate':candidates[year,'REPUBLICAN'],
        'party_detailed':'REPUBLICAN',
        'candidatevotes':rep_votes,
        'totalvotes':ny_votes[ny_votes.year==year].totalvotes.iloc[0],
        'party_simplified':'REPUBLICAN'
    })

In [106]:
# convert list to DataFrame
presel_ny=pd.DataFrame(ny_total_votes_list)
presel_ny

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
0,1976,NEW YORK,NY,36,21,13,"CARTER, JIMMY",DEMOCRAT,3389558,6668262,DEMOCRAT
1,1976,NEW YORK,NY,36,21,13,"FORD, GERALD",REPUBLICAN,3100791,6668262,REPUBLICAN
2,1980,NEW YORK,NY,36,21,13,"CARTER, JIMMY",DEMOCRAT,2728372,6201959,DEMOCRAT
3,1980,NEW YORK,NY,36,21,13,"REAGAN, RONALD",REPUBLICAN,2893831,6201959,REPUBLICAN
4,1984,NEW YORK,NY,36,21,13,"MONDALE, WALTER",DEMOCRAT,3119609,6806810,DEMOCRAT
5,1984,NEW YORK,NY,36,21,13,"REAGAN, RONALD",REPUBLICAN,3664763,6806810,REPUBLICAN
6,1988,NEW YORK,NY,36,21,13,"DUKAKIS, MICHAEL",DEMOCRAT,3347882,6485683,DEMOCRAT
7,1988,NEW YORK,NY,36,21,13,"BUSH, GEORGE H.W.",REPUBLICAN,3081871,6485683,REPUBLICAN
8,1992,NEW YORK,NY,36,21,13,"CLINTON, BILL",DEMOCRAT,3444450,7079432,DEMOCRAT
9,1992,NEW YORK,NY,36,21,13,"BUSH, GEORGE H.W.",REPUBLICAN,2346649,7079432,REPUBLICAN


Now let's delete New York's data from the original DataFrame and merge two DataFrames to get the appropriate dataset.

In [107]:
presel=presel[presel.state_po!='NY']
presel=pd.concat([presel,presel_ny],axis=0,ignore_index=True)

In [108]:
presel

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
0,1976,ALABAMA,AL,1,63,41,"CARTER, JIMMY",DEMOCRAT,659170,1182850,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,"FORD, GERALD",REPUBLICAN,504070,1182850,REPUBLICAN
2,1976,ALASKA,AK,2,94,81,"FORD, GERALD",REPUBLICAN,71555,123574,REPUBLICAN
3,1976,ALASKA,AK,2,94,81,"CARTER, JIMMY",DEMOCRAT,44058,123574,DEMOCRAT
4,1976,ARIZONA,AZ,4,86,61,"FORD, GERALD",REPUBLICAN,418642,742719,REPUBLICAN
...,...,...,...,...,...,...,...,...,...,...,...
1219,2012,NEW YORK,NY,36,21,13,"ROMNEY, MITT",REPUBLICAN,2485432,7116784,REPUBLICAN
1220,2016,NEW YORK,NY,36,21,13,"CLINTON, HILLARY",DEMOCRAT,4556124,7802084,DEMOCRAT
1221,2016,NEW YORK,NY,36,21,13,"TRUMP, DONALD J.",REPUBLICAN,2819534,7802084,REPUBLICAN
1222,2020,NEW YORK,NY,36,21,13,"BIDEN, JOSEPH R. JR",DEMOCRAT,5230985,8661735,DEMOCRAT


In [109]:
presel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              1224 non-null   int64 
 1   state             1224 non-null   object
 2   state_po          1224 non-null   object
 3   state_fips        1224 non-null   int64 
 4   state_cen         1224 non-null   int64 
 5   state_ic          1224 non-null   int64 
 6   candidate         1224 non-null   object
 7   party_detailed    1224 non-null   object
 8   candidatevotes    1224 non-null   int64 
 9   totalvotes        1224 non-null   int64 
 10  party_simplified  1224 non-null   object
dtypes: int64(6), object(5)
memory usage: 105.3+ KB


Let's look at the party_detailed and party_simplified columns again

In [110]:
presel.party_detailed.value_counts()

party_detailed
REPUBLICAN                 612
DEMOCRAT                   609
DEMOCRATIC-FARMER-LABOR      3
Name: count, dtype: int64

In [111]:
presel.party_simplified.value_counts()

party_simplified
DEMOCRAT      612
REPUBLICAN    612
Name: count, dtype: int64

We've got the correct number: one democrat and one republican for each of the elections.
There were ((2020-1976)/4 + 1)*(50 states + 1 DC) = 12 * 51 = 612 elections.

And just to make sure, let's see the rows where their values are not equal...

In [112]:
presel[presel.party_detailed!=presel.party_simplified]

,year,state,state_po,state_fips,state_cen,state_ic,candidate,party_detailed,candidatevotes,totalvotes,party_simplified
646,2000,MINNESOTA,MN,27,41,33,"GORE, AL",DEMOCRATIC-FARMER-LABOR,1168266,2438685,DEMOCRAT
746,2004,MINNESOTA,MN,27,41,33,"KERRY, JOHN",DEMOCRATIC-FARMER-LABOR,1445014,2828387,DEMOCRAT
946,2012,MINNESOTA,MN,27,41,33,"OBAMA, BARACK H.",DEMOCRATIC-FARMER-LABOR,1546167,2936561,DEMOCRAT


We can drop the 'party_detailed' column and rename 'party_simplified' to 'party'

In [ ]:
presel.drop(columns=['party_detailed'],inplace=True)

In [115]:
presel.rename(columns={'party_simplified':'party'},inplace=True)

In [116]:
presel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            1224 non-null   int64 
 1   state           1224 non-null   object
 2   state_po        1224 non-null   object
 3   state_fips      1224 non-null   int64 
 4   state_cen       1224 non-null   int64 
 5   state_ic        1224 non-null   int64 
 6   candidate       1224 non-null   object
 7   candidatevotes  1224 non-null   int64 
 8   totalvotes      1224 non-null   int64 
 9   party           1224 non-null   object
dtypes: int64(6), object(4)
memory usage: 95.8+ KB


Making sure we have exactly 1 entry for each year, party and state

In [125]:
(presel.groupby(['year','party','state']).candidate.count().values==np.ones(1224,)).any()

True

Let's sort the data by year and state_fips, as it was originally, and save it in a file.

In [127]:
presel.sort_values(by=['year','state_fips'],inplace=True)

In [128]:
presel.to_csv('data/cleaned_1976-2020-president.csv',index=False)